<a href="https://colab.research.google.com/github/Deepak202011/Deepak202011/blob/main/PoseGuard_Day1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mediapipe==0.10.14 opencv-python -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 11.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.


In [2]:
import mediapipe as mp
print("✅ MediaPipe imported!")
print("Key attributes:", [attr for attr in dir(mp) if 'solution' in attr.lower() or 'pose' in attr.lower()])

# Test solutions
try:
    mp.solutions.pose
    print("✅ mp.solutions.pose → READY!")
except:
    print("❌ Still broken - restart runtime")



✅ MediaPipe imported!
Key attributes: ['solutions']
✅ mp.solutions.pose → READY!


In [4]:
!pip install gTTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.1
    Uninstalling click-8.3.1:
      Successfully uninstalled click-8.3.1


In [5]:
import cv2
import mediapipe as mp
import numpy as np
from gtts import gTTS
from IPython.display import Audio
import io



from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5
)

def extract_stress_fetaures(results):
  if results.pose_landmarks:
    landmarks = results.pose_landmarks.landmark
     #stress signal 1
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    shoulder_drop = abs(left_shoulder.y - right_shoulder.y)

    #stress signal 2
    nose = landmarks[mp_pose.PoseLandmark.NOSE.value]
    left_ear = landmarks[mp_pose.PoseLandmark.LEFT_EAR.value]
    ##head_tilt = landmarks[mp_pose.PoseLandmark.HEAD_TILT.value]
    right_ear = landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value]
    head_tilt= abs(left_ear.y - right_ear.y) * 100
    nose_dive = abs(nose.y - (left_ear.y - right_ear.y)/2) * 100

    #stress score
    stress_score = (head_tilt + nose_dive)/2
    return stress_score
  return 0.0
print("Poseguard stress analyzer is ready.")

Poseguard stress analyzer is ready.


In [6]:
from gtts import gTTS
from IPython.display import Audio
tts = gTTS("Stress detection test complete")
tts.save("test.mp3")
Audio("test.mp3")

In [7]:
import warnings
warnings.filterwarnings("ignore", message="SymbolDatabase.GetPrototype")


In [ ]:
print([name for name in dir(mp_pose.PoseLandmark) if 'EAR' in name or 'NOSE' in name])



In [35]:
def take_photo():
    js = Javascript('''
        async function takePhoto() {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Take Photo';
            div.appendChild(capture);
            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();
            await new Promise((resolve) => capture.onclick = resolve);
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', 0.8);
        }
    ''')
    display(js)
    data = eval_js('takePhoto()')
    binary = b64decode(data.split(',')[1])
    with open('photo.jpg', 'wb') as f:
        f.write(binary)
    print("Photo saved!")
    return 'photo.jpg'


image_path = take_photo()
image = cv2.imread(image_path)
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

results = pose.process(rgb_image)
stress = extract_stress_fetaures(results)

def speak_stress_guidance(stress_score):
    if stress_score > 40:
        text = "High stress detected. Take deep breaths. Inhale for four. Hold for four. Exhale for four."
    elif stress_score > 25:
        text = "Moderate stress. Breathe slowly. Relax your shoulders."
    else:
        return

    # Generate speech
    tts = gTTS(text=text, lang='en', slow=False)
    fp = io.BytesIO()
    tts.write_to_fp(fp)
    fp.seek(0)

    # Plays in Colab
    display(Audio(fp.read(), autoplay=True))


print(f"STRESS SCORE: {stress:.2f}")
speak_stress_guidance(stress)





<IPython.core.display.Javascript object>

Photo saved!
STRESS SCORE: 26.94


In [10]:
results = pose.process(rgb_image)
stress = extract_stress_fetaures(results)

print(f"STRESS SCORE: {stress:.2f}", end=" | ")
if stress > 40:
    print("HIGH STRESS - Speaking...")
elif stress > 25:
    print("MODERATE STRESS - Speaking...")
else:
    print("RELAXED")

speak_stress_guidance(stress)


STRESS SCORE: 32.28 | MODERATE STRESS - Speaking...


In [38]:
import pandas as pd
stress_dataset = []
def log_stress_data(head_tilt, nose_dive, shoulder_drop, label):
  data = {
      'head_tilt': head_tilt,
      'nose_dive': nose_dive,
      'shoulder_drop': shoulder_drop,
      'label': label
  }
  stress_dataset.append(data)
  df =pd.DataFrame(stress_dataset)
  df.to_csv('stress_dataset.csv', index=False)
  print(f"Collected {len(stress_dataset)} samples")

In [12]:
def extract_stress_features(results):
    if not results.pose_landmarks:
        return 0.0

    landmarks = results.pose_landmarks.landmark

    # Extract 3 key stress features
    head_tilt = abs(landmarks[7].y - landmarks[8].y) * 100      # LEFT_EAR - RIGHT_EAR
    nose_dive = abs(landmarks[0].y - (landmarks[7].y + landmarks[8].y)/2) * 100  # NOSE position
    shoulder_drop = abs(landmarks[11].y - landmarks[12].y) * 100  # LEFT_SHOULDER - RIGHT_SHOULDER

    # Auto-label using your current rules
    stress_score = (head_tilt + nose_dive + shoulder_drop) / 3
    label = 'high' if stress_score > 40 else 'medium' if stress_score > 25 else 'low'

    # SAVE DATA AUTOMATICALLY
    log_stress_data(head_tilt, nose_dive, shoulder_drop, label)

    return stress_score


In [40]:
from sklearn.ensemble import RandomForestClassifier
import joblib

df = pd.read_csv('stress_dataset.csv')
print(f"Training on {len(df)} samples")
print(df.head())

X = df[['head_tilt', 'nose_dive', 'shoulder_drop']]
y = df['label']

model = RandomForestClassifier(n_estimators=100)
model.fit(X, y)

joblib.dump(model, 'stress_model.pkl')
print("Model trained and saved!")

FileNotFoundError: [Errno 2] No such file or directory: 'stress_dataset.csv'